In [1]:
import pandas as pd
import numpy as np

train = pd.read_csv("/kaggle/input/identify-the-sentiment-av/train_2kmZucJ.csv")

train.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


In [2]:
from sklearn.model_selection import train_test_split

train, valid = train_test_split(train, test_size=0.2, random_state=42, stratify=train['label'])

In [3]:
Model_Name = "HuggingFaceTB/SmolLM-360M"

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model = AutoModelForSequenceClassification.from_pretrained(Model_Name, num_labels=2)

tokenizer = AutoTokenizer.from_pretrained(Model_Name)

config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

2025-07-23 07:40:40.752007: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753256440.951586      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753256441.011422      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/1.45G [00:00<?, ?B/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at HuggingFaceTB/SmolLM-360M and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/831 [00:00<?, ?B/s]

In [5]:
from datasets import Dataset

COLS = ['tweet', 'label']

train_df_clean = train[COLS].copy()
val_df_clean = valid[COLS].copy()

# Ensure labels are proper integers
train_df_clean['label'] = train_df_clean['label'].astype(np.int64)
val_df_clean['label'] = val_df_clean['label'].astype(np.int64)

# Reset index to ensure clean DataFrame structure
train_df_clean = train_df_clean.reset_index(drop=True)
val_df_clean = val_df_clean.reset_index(drop=True)

# Create datasets with explicit copy to avoid NumPy 2.0 issues
train_ds = Dataset.from_pandas(train_df_clean, preserve_index=False)
val_ds = Dataset.from_pandas(val_df_clean, preserve_index=False)

In [6]:
def tokenize(batch):
    """Tokenizes a batch of text inputs."""
    return tokenizer(batch["tweet"], truncation=True, max_length=256)

# Apply tokenization
train_ds = train_ds.map(tokenize, batched=True, remove_columns=['tweet'])
val_ds = val_ds.map(tokenize, batched=True, remove_columns=['tweet'])

Map:   0%|          | 0/6336 [00:00<?, ? examples/s]

Map:   0%|          | 0/1584 [00:00<?, ? examples/s]

In [7]:
# Add a new padding token

tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Resize the model's token embeddings to match the new tokenizer

model.resize_token_embeddings(len(tokenizer))

# Set the pad token id in the model's config

model.config.pad_token_id = tokenizer.pad_token_id

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [8]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding

DIR = "/kaggle/working/"
# --- Training Arguments ---
training_args = TrainingArguments(
    output_dir=f"./{DIR}",
    do_train=True,
    do_eval=True,
    eval_strategy="steps", # Evaluate every 'eval_steps'
    save_strategy="steps", # Save model every 'save_steps'
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=24,
    learning_rate=1e-5,
    logging_dir="./logs",
    logging_steps=500,
    save_steps=500,
    eval_steps=500,
    weight_decay=0.05,
    save_total_limit=1, # Only save the best model
    metric_for_best_model="accuracy",
    greater_is_better=True,
    load_best_model_at_end=True, # Load the best model found during training
    report_to="none", # Do not report to external services like Weights & Biases
    warmup_ratio=0.1, # 10% of total steps will be used for linear warmup
    lr_scheduler_type="cosine", # Use cosine learning rate decay
    dataloader_pin_memory=False, # Disable pin memory to avoid potential issues
)

In [9]:
from sklearn.metrics import accuracy_score, roc_auc_score
from scipy.special import softmax

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = softmax(logits, axis=-1)[:, 1]  # Class 1 probability
    preds = logits.argmax(axis=-1)
    
    acc = accuracy_score(labels, preds)
    auc = roc_auc_score(labels, probs)
    
    return {
        "accuracy": acc,
        "roc_auc": auc
    }



data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        processing_class=tokenizer,
        compute_metrics=compute_metrics,
        data_collator=data_collator, # Use dynamic padding
    )
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,Roc Auc
500,0.358300,0.248539,0.897096,0.950772
1000,0.172900,0.257416,0.905934,0.957891
1500,0.103100,0.297329,0.905934,0.956552


TrainOutput(global_step=1980, training_loss=0.17558844884236655, metrics={'train_runtime': 1314.1214, 'train_samples_per_second': 24.107, 'train_steps_per_second': 1.507, 'total_flos': 5732789010278400.0, 'train_loss': 0.17558844884236655, 'epoch': 5.0})

In [10]:
test = pd.read_csv("/kaggle/input/identify-the-sentiment-av/test_oJQbWVk.csv")

ds_test = Dataset.from_pandas(test[['tweet']])
ds_test = ds_test.map(tokenize, batched=True)

Map:   0%|          | 0/1953 [00:00<?, ? examples/s]

In [11]:
predictions = trainer.predict(ds_test)
probs = torch.nn.functional.softmax(torch.tensor(predictions.predictions), dim=1).numpy()

In [ ]:
predicted_labels = probs.argmax(axis=1)
submission = pd.DataFrame({
    "id": test["id"],
    "label": predicted_labels 
})

submission.to_csv("submission.csv", index=False)

In [13]:
trainer.save_model("/kaggle/working/Best_model")